In [2]:
from pynq import Overlay
from pynq import allocate
import numpy as np

In [3]:
ol =Overlay('/home/xilinx/pynq/overlays/Radix2wECC/design_1.bit')

In [4]:
my_ip = ol.Radix2wECC_0
#my_ip.register_map
#ol.ip_dict
my_ip.register_map.CTRL.AP_DONE

0

In [5]:
k = np.array([0x00000005, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000], dtype=np.int32)
k_buffer = allocate(shape=(6,), dtype=np.int32)
#_buffer = allocate(shape=(6,), dtype=np.int32)
#y_buffer = allocate(shape=(6,), dtype=np.int32)
np.copyto(k_buffer,k)
my_ip.register_map.k_1 = k_buffer.physical_address
#my_ip.register_map.x_1 = x_buffer.physical_address
#my_ip.register_map.y_1 = y_buffer.physical_address
#print(hex(k_buffer.physical_address))
#print(hex(my_ip.register_map.k_1.address))
#print(hex(my_ip.register_map.k_2.address))
print(my_ip.register_map.CTRL.AP_IDLE)
my_ip.register_map.CTRL.START=1
print(my_ip.register_map.CTRL.AP_IDLE)


1
1


In [6]:
my_ip.register_map.CTRL.AP_DONE

0

In [7]:
k_np = np.frombuffer(k_buffer.data, dtype=np.int32)
print(k_np)


[5 0 0 0 0 0]


In [8]:
x_np = np.frombuffer(x_buffer.data, dtype=np.int32)
print(x_np)


NameError: name 'x_buffer' is not defined

In [ ]:
y_np = np.frombuffer(y_buffer.data, dtype=np.int32)
print(y_np)
